# MLOps or how not to die to put models in production

## MLFlow Models
In this section, we can see an example of using a trained model from MLFlow experiments.

### Load a trained model

In [ ]:
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

After decide which model will be used, paste here both experiment_id and run_id.

In [ ]:
experiment_id = "<EXPERIMENT_ID>"
run_id = "<RUN_ID>"

In [ ]:
cancer = load_breast_cancer()
X = np.array(cancer.data)
y = np.array(cancer.target)

In [ ]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=426, test_size=143, random_state=0)
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
sel = np.random.randint(len(x_test))
x_sel = x_test[sel]
y_sel = y_test[sel]
print(y_sel)

In [ ]:
model = mlflow.sklearn.load_model(f'../mlruns/{experiment_id}/{run_id}/artifacts/model')

In [ ]:
model.predict([x_sel])

## Create a docker image to put in production

In [ ]:
! mlflow models build-docker -m '<MODEL_FILE_PATH>' -n "<IMAGE_NAME>"

Run the container

In [ ]:
! docker run -it -p 6001:8080 "<IMAGE_NAME>"

Create a function to request to the microservice

In [ ]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

Input data for the request

In [ ]:
query_input = {'data': [x_sel.tolist()]}
query_input

Make the inference to the microservice

In [ ]:
prod_prediction1 = query_endpoint_example(scoring_uri='http://127.0.0.1:6001/invocations', inputs=query_input)